In [1]:
import numpy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pytz
import geopandas as gpd
from datetime import datetime, timedelta
import scipy.io as sio
from tqdm import tqdm
import os

from coastsat_keras import *

transect_data = gpd.read_file('data_coastsat/transects_edit.geojson')
transect_data = transect_data[transect_data['id'].str.contains("usa_CA")]
def linestring_to_points(feature,line):
    return line.coords

transect_data['points'] = transect_data.apply(lambda l: linestring_to_points(l['site_id'],l['geometry']), axis=1)
site_id = transect_data['id'].to_numpy()

shore_data_new = pd.read_csv('extended_coastsat_data/All_CA_April_2023.csvx')

sla_transect_mat = sio.loadmat('sla/sla_transect')
sla_transect = np.squeeze(sla_transect_mat['sla_transect'])
time_sla = np.squeeze(sla_transect_mat['time_sla'])

In [ ]:
j = 0
llon_1 = [0 for i in range(len(transect_data['id']))] 
llon_2 = [0 for i in range(len(transect_data['id']))] 
orient = [0 for i in range(len(transect_data['id']))] 
mean_train = [np.NaN for i in range(len(transect_data['id']))] 
std_train = [np.NaN for i in range(len(transect_data['id']))] 
mse = [np.NaN for i in range(len(transect_data['id']))] 
slope = [np.NaN for i in range(len(transect_data['id']))] 

N = 500
slope_pred = np.zeros((N,len(transect_data['id'])))*np.NaN
fout = '/Users/suadusumilli/sioglaciology Dropbox/Susheel Adusumilli/code/coastsat/coastsat_keras_adam_mse/'

for tid in tqdm(transect_data['id']):
    tid_ = transect_data[transect_data['id'].str.contains(tid)]['points']
    for tid__ in tid_:
        llon_1[j] = tid__[0]
        llon_2[j] = tid__[1]
    oid_ = transect_data[transect_data['id'].str.contains(tid)]['orientation']
    for oid__ in oid_:
        orient[j] = oid__
        
    shore_data,tide_heights,wave_data,dist_mop,spec1d_interp,freqz = read_shore_data_new(tid,site_id,shore_data_new)
    if dist_mop>400:
        # 400 for the first pass only, edit it down to 200 for post-processing
        j+=1
        continue
    if not isinstance(shore_data, pd.DataFrame):
        continue    
    df, df_time = create_df_for_keras(shore_data,tide_heights,wave_data,spec1d_interp,freqz,tid)
    X_all,X_train,X_val,X_test,y_all,y_train,y_val,y_test,df_mean,df_std = create_train_val_test_data_no_proj(df)
    
    X_tide = np.zeros((N,np.shape(X_all)[1]))
    X_tide[:,0] = np.linspace(np.nanmin(X_all[:,0]),np.nanmax(X_all[:,0]),np.shape(X_tide[:,0])[0])

    n_features = X_train.shape[1]

    model = Sequential()
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform', input_shape=(n_features,)))
    model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    # compile the model
    # main keras models are using rmsprop/mse, adam/mae in the first instance here is a test run
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=200, batch_size=np.shape(X_train)[0], verbose=0, validation_data=(X_val, y_val))
    model.save(fout + 'keras_model_' + str(j) + '.h5')
    clear_session()
    j+=1